In [3]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import sk_plotting_functions as spf
sys.path.insert(1, '../')
import SIKE
sys.path.insert(1, '/Users/dpower/Documents/01 - PhD/01 - Code/08 - SIKE/legacy')
import SIKE1
from post_processing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Compare with ADAS

In [4]:
el = 'He'
num_x = 100
xgrid = np.linspace(0,10,num_x)
ne=1e19 * np.ones(num_x)
Te=np.geomspace(1.0,100,num_x)

In [ ]:
# SIKE 1 run
sike_opts = {'EVOLVE': False,
                'SAVE': False,
                'SAVE_PATH': 'SIKE1_' + el,
                'MODELLED SPECIES': [el],
                'MODELLED_STATES': 'ground',
                'DELTA_T': 1.0e8,
                'RES_THRESH': 1e-12,
                'MAX_STEPS': 1e5,
                'T_SAVE': 1e6,
                'FRAC_IMP_DENS': 0.05,
                'COLL_ION_REC': True,
                'RAD_REC': True,
                'COLL_EX_DEEX': True,
                'SPONT_EM': False,
                'COMPARE_ADAS': True}
sp = SIKE1.run(Te=Te,ne=ne,opts=sike_opts)
r1 = sp[el]

In [6]:
# SIKE 2 run
r2 = SIKE.SIKERun(xgrid=xgrid, Te=Te, ne=ne, opts={'modelled_impurities': [el], 
                                                            'evolve': True, 
                                                            'ionization': True,
                                                            'radiative recombination': True,
                                                            'excitation': True,
                                                            'resolve_j': True,
                                                            'emission': True,
                                                            'autoionization': True,
                                                            'delta_t': 1000000.0,
                                                            'max_steps': 1000, 
                                                            'dndt_thresh': 1e16,
                                                            'state_ids': None})
r2.run()
# r2.build_matrix()
# r2.compute_densities(r2.opts['delta_t'],r2.opts['max_steps'], r2.opts['evolve'])

Initialising the impurity species to be modelled...
 Initialising states...
 Initialising transitions...
  Loading transitions from json...
  Creating transition objects...
  Creating data for inverse transitions...
  Performing checks on transition data...
 Initialising densities...
 Finalising states...
Finished initialising impurity species objects.
Filling Maxwellian transition matrix for He...
 100.0%
Computing densities with Maxwellian electrons for He...
Evolving with: ibcgs , preconditioner:  bjacobi
TIMESTEP 1 | max(dn/dt) 5.00e+11 | NUM_ITS 63
Conservation check on rank 0: 1.97e-07


In [ ]:
r2.compute_densities(r2.opts['delta_t']/100,r2.opts['max_steps'], r2.opts['evolve'])

In [ ]:
print('Total evolved states (j-resolved): ' + str(r2.impurities[el].tot_states))
print('Total transitions (j-resolved): ' + str(len(r2.impurities[el].transitions)))

In [ ]:
# Compare Zavg
r1.get_Zavg()
Zavg_1 = r1.Zavg_adas
Zavg_2_Max = get_Zavg(r2.impurities[el].dens_Max, r2.impurities[el].states, r2.num_x)

fig,ax = plt.subplots(1)
l,=ax.plot(Te,Zavg_1, '-.', color='black', label='ADAS')
ax.plot(Te,Zavg_2_Max, color=l.get_color(), label='FAC (Max)')
ax.legend()
ax.set_xscale('log')
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Average ionization')
ax.set_title('Average ionization: ' +el)

In [ ]:
# Compare Z dens
num_Z = r1.num_z
Z_dens_1 = r1.get_z_dens_adas()
Z_dens_2_Max = get_Z_dens(r2.impurities[el].dens_Max, r2.impurities[el].states)

fig,ax = plt.subplots(1)
for Z in range(num_Z):
    l, = ax.plot(Te, Z_dens_2_Max[:,Z]*r2.n_norm, label=el + '$^{' + str(Z) + '+}$')
    ax.plot(Te, Z_dens_1[:,Z]*r1.n_norm,'-.',color=l.get_color())
ax.plot([],[],color='black', label='FAC (Max)')
ax.plot([],[],'-.',color='black', label='ADAS')
ax.legend()
ax.set_xscale('log')
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Density [m$^{-3}$]')
ax.set_title('Density profiles per ionization stage: ' + el)

In [ ]:
# Plot PLTs
PLT_1, _, _, PLT_1_eff = r1.get_adas_PLT()
PLT_2_Max, PLT_2_Max_eff = get_cooling_curves(r2, el, kinetic=False)
fig,ax = plt.subplots(1)
for Z in range(r1.num_z-1):
    l, = ax.plot(Te,PLT_2_Max[:,Z],label=el + '$^{' + str(Z) + '+}$')
    ax.plot(Te,PLT_1[:,Z],'-.',color=l.get_color())
ax.plot(Te,PLT_2_Max_eff,'-',color='black', label='Effective (FAC)')
ax.plot(Te,PLT_1_eff,'-.',color='black', label='Effective (ADAS)')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Excitation radiation per ion [Wm$^3$]')
# ax.set_ylim([1e-39,1e-30])
ax.legend()
ax.grid()
ax.set_title('Cooling curves: ' + el)

In [ ]:
# Plot radiative losses
PLT_1, _, _, PLT_1_eff = r1.get_adas_PLT()
Q_rad_1 = 1e-6 * PLT_1_eff * ne * np.sum(r1.dens_adas,1) * r1.n_norm
PLT_2_Max, PLT_2_Max_eff = get_cooling_curves(r2, el, kinetic=False)
Q_rad_2_Max = 1e-6 * PLT_2_Max_eff * ne * np.sum(r2.impurities[el].dens_Max,1) * r2.n_norm
fig,ax = plt.subplots(1)
l, = ax.plot(Te,Q_rad_2_Max,color='black',label='FAC (Max)')
ax.plot(Te,Q_rad_1,'-.',color=l.get_color(), label='ADAS')
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Excitation radiation [Wm$^{-3}$]')
ax.legend()
ax.grid()
ax.set_title('Total line radiation profiles: ' + el)
fig.tight_layout()

# Compare fluid vs. kinetic

In [ ]:
el = 'H'
P_in = 64    # Input power [MW/m^2]
n_avg = 10
skip = 20
max_x = None
# run_dir = '/Users/dpower/Documents/01 - PhD/14 - ELM investigation/01 - Runs/01 - Equilibria/02 - Kinetic/P_in = ' + str(P_in) + 'MW/Output_job_EQ_K' + str(P_in) + '_' + str(n_avg) + 'e19'
run_dir = '/Users/dpower/Documents/01 - PhD/15 - Reactor-relevant runs/02 - Kinetic/Output_RR_K_18e19'
sktrun = spf.SKTRun(run_dir)
xgrid = sktrun.xgrid[:max_x:skip]
sktrun.load_dist()
fe = sktrun.data['DIST_F'][0][:,:max_x:skip] * sktrun.n_norm / sktrun.v_th ** 3
vgrid = sktrun.vgrid
Te = sktrun.data['TEMPERATURE'][:max_x:skip] * sktrun.T_norm
ne = sktrun.data['DENSITY'][:max_x:skip] * sktrun.n_norm

In [ ]:
# SIKE 1 run
sike_opts = {'EVOLVE': False,
                'SAVE': False,
                'SAVE_PATH': 'SIKE1_' + el,
                'MODELLED SPECIES': [el],
                'MODELLED_STATES': 'ground',
                'DELTA_T': 1.0e8,
                'RES_THRESH': 1e-12,
                'MAX_STEPS': 1e5,
                'T_SAVE': 1e6,
                'FRAC_IMP_DENS': 0.05,
                'COLL_ION_REC': True,
                'RAD_REC': True,
                'COLL_EX_DEEX': True,
                'SPONT_EM': False,
                'COMPARE_ADAS': True}
sp = SIKE1.run(sktrun,opts=sike_opts)
r1 = sp[el]

In [ ]:
# SIKE 2 run
r2 = SIKE.SIKERun(xgrid=xgrid, fe=fe, vgrid=vgrid, opts={'modelled_impurities': [el], 
                                                          'kinetic_electrons': True,
                                                          'maxwellian_electrons': True,
                                                          'evolve': False, 
                                                          'resolve_j': True,
                                                          'ionization': True,
                                                          'radiative recombination': True,
                                                          'excitation': True,
                                                          'emission': True,
                                                          'autoionization': True,
                                                          'delta_t': 10000.0,
                                                          'max_steps': 10000, 
                                                          'frac_imp_dens': 1.0,
                                                          'dndt_thresh': 1e16,
                                                          'state_ids': None})
r2.run()
r2.build_matrix()
r2.compute_densities(r2.opts['delta_t'],r2.opts['max_steps'], r2.opts['evolve'])

In [ ]:
print('Total evolved states: ' + str(r2.impurities[el].tot_states))
print('Total transitions: ' + str(len(r2.impurities[el].transitions)))

In [ ]:
# Compare Zavg
r1.get_Zavg()
# Zavg_1 = r1.Zavg_adas
Zavg_2_Max = get_Zavg(r2.impurities[el].dens_Max, r2.impurities[el].states, r2.num_x)
Zavg_2_kin = get_Zavg(r2.impurities[el].dens, r2.impurities[el].states, r2.num_x)

fig,ax = plt.subplots(1)
l, = ax.plot(Te,Zavg_2_Max, color='black', label='FAC (Max)')
ax.plot(Te,Zavg_2_kin, '--', color=l.get_color(),label='FAC (kin)')
ax.legend()
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Average ionization')
ax.set_title('Average ionization: ' +el)

In [ ]:
# Compare Z dens
num_Z = r1.num_z
Z_dens_2_Max = get_Z_dens(r2.impurities[el].dens_Max, r2.impurities[el].states)
Z_dens_2_kin = get_Z_dens(r2.impurities[el].dens, r2.impurities[el].states)

fig,ax = plt.subplots(1)
for Z in range(num_Z):
    l, = ax.plot(Te, Z_dens_2_Max[:,Z]*r2.n_norm, label=el + '$^{' + str(Z) + '+}$')
    ax.plot(Te, Z_dens_2_kin[:,Z]*r2.n_norm, '--', color=l.get_color())
ax.plot([],[],color='black', label='FAC (Max)')
ax.plot([],[],'--',color='black', label='FAC (kin)')
ax.legend()
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Density [m$^{-3}$]')
ax.set_title('Density profiles per ionization stage: ' + el)

In [ ]:
# Plot PLTs
PLT_2_Max, PLT_2_Max_eff = get_cooling_curves(r2, el, kinetic=False)
PLT_2_kin, PLT_2_kin_eff = get_cooling_curves(r2, el, kinetic=True)
fig,ax = plt.subplots(1)
for Z in range(r1.num_z-1):
    l, = ax.plot(Te,PLT_2_Max[:,Z],label=el + '$^{' + str(Z) + '+}$')
    ax.plot(Te,PLT_2_kin[:,Z],'--',color=l.get_color())
# ax.plot(Te,PLT_2_Max_eff,'-',color='black', label='FAC (Max)')
# ax.plot(Te,PLT_2_kin_eff,'--',color='black', label='FAC (kin)')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Excitation radiation per ion [Wm$^3$]')
ax.legend()
ax.grid()
ax.set_title('Cooling curves: ' + el)

In [ ]:
# Plot radiative losses
PLT_2_Max, PLT_2_Max_eff = get_cooling_curves(r2, el, kinetic=False)
Q_rad_2_Max = 1e-6 * PLT_2_Max_eff * ne * np.sum(r2.impurities[el].dens_Max,1) * r2.n_norm
PLT_2_kin, PLT_2_kin_eff = get_cooling_curves(r2, el, kinetic=True)
Q_rad_2_kin = 1e-6 * PLT_2_kin_eff * ne * np.sum(r2.impurities[el].dens,1) * r2.n_norm
fig,ax = plt.subplots(1)
l, = ax.plot(r1.xgrid,Q_rad_2_Max,color='black',label='FAC (Max)')
ax.plot(r1.xgrid,Q_rad_2_kin,'--',color=l.get_color(), label='FAC (kin)')
ax.set_xlabel('x [m]')
ax.set_ylabel('Excitation radiation [Wm$^{-3}$]')
ax.legend()
ax.grid()
ax.set_title('Total line radiation profiles: ' + el)

# Compare j-resolved vs. aggregated

In [ ]:
el = 'Li'
num_x = 50
xgrid = np.linspace(0,1,num_x)
ne=1e19 * np.ones(num_x)
Te=np.geomspace(0.1,10,num_x)

In [ ]:
# j-resolved
r_nlj = SIKE2.SIKERun(xgrid=xgrid, Te=Te, ne=ne, opts={'modelled_impurities': [el], 
                                                            'evolve': True, 
                                                            'ionization': True,
                                                            'radiative recombination': True,
                                                            'excitation': True,
                                                            'resolve_j': True,
                                                            'emission': True,
                                                            'autoionization': True,
                                                            'delta_t': 100000.0,
                                                            'max_steps': 1000, 
                                                            'dndt_thresh': 1e17,
                                                            'state_ids': None})
r_nlj.build_matrix()
r_nlj.compute_densities(r_nlj.opts['delta_t'],r_nlj.opts['max_steps'], r_nlj.opts['evolve'])

In [ ]:
# j-aggregated
r_nl = SIKE2.SIKERun(xgrid=xgrid, Te=Te, ne=ne, opts={'modelled_impurities': [el], 
                                                            'evolve': True, 
                                                            'ionization': True,
                                                            'radiative recombination': True,
                                                            'excitation': True,
                                                            'resolve_j': False,
                                                            'emission': True,
                                                            'autoionization': True,
                                                            'delta_t': 100000.0,
                                                            'max_steps': 1000, 
                                                            'dndt_thresh': 1e17,
                                                            'state_ids': None})
r_nl.run()
# r_nl.build_matrix(kinetic=False)
# r_nl.compute_densities(r_nl.opts['delta_t'],r_nl.opts['max_steps'], r_nl.opts['evolve'],kinetic=False)

In [ ]:
print('Total evolved states, j-resolved: ' + str(r_nlj.impurities[el].tot_states) + ', j-aggregated: ' + str(r_nl.impurities[el].tot_states))
print('Total transitions, j-resolved: ' + str(len(r_nlj.impurities[el].transitions)) + ', j-aggregated: ' + str(len(r_nl.impurities[el].transitions)))

In [ ]:
# Compare Zavg
Zavg_nlj = get_Zavg(r_nlj.impurities[el].dens_Max, r_nlj.impurities[el].states, r_nlj.num_x)
Zavg_nl = get_Zavg(r_nl.impurities[el].dens_Max, r_nl.impurities[el].states, r_nl.num_x)

fig,ax = plt.subplots(1)
l, = ax.plot(Te,Zavg_nlj, color='red', label='j-resolved')
ax.plot(Te,Zavg_nl, '--', color='blue',label='j-aggregated')
ax.legend()
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Average ionization')
ax.set_title('Average ionization: ' +el)

In [ ]:
# Compare Z dens
num_Z = r_nlj.impurities[el].num_Z
Z_dens_nlj = get_Z_dens(r_nlj.impurities[el].dens_Max, r_nlj.impurities[el].states)
Z_dens_nl = get_Z_dens(r_nl.impurities[el].dens_Max, r_nl.impurities[el].states)

fig,ax = plt.subplots(1)
for Z in range(num_Z):
    l, = ax.plot(Te, Z_dens_nlj[:,Z]*r_nlj.n_norm, label=el + '$^{' + str(Z) + '+}$')
    ax.plot(Te, Z_dens_nl[:,Z]*r_nl.n_norm, '--', color=l.get_color())
ax.plot([],[],color='black', label='j-resolved')
ax.plot([],[],'--',color='black', label='j-aggregated')
ax.legend()
ax.set_xlabel('x [m]')
ax.set_ylabel('Density [m$^{-3}$]')
ax.set_title('Density profiles per ionization stage: ' + el)

In [ ]:
# Plot PLTs
PLT_nlj, PLT_nlj_eff = get_cooling_curves(r_nlj, el, kinetic=False)
PLT_nl, PLT_nl_eff = get_cooling_curves(r_nl, el, kinetic=False)
fig,ax = plt.subplots(1)
for Z in range(r_nlj.impurities[el].num_Z-1):
    l, = ax.plot(Te,PLT_nlj[:,Z],label=el + '$^{' + str(Z) + '+}$')
    ax.plot(Te,PLT_nl[:,Z],'--',color=l.get_color())
ax.plot(Te,PLT_nlj_eff,'-',color='black', label='j-resolved')
ax.plot(Te,PLT_nl_eff,'--',color='black', label='j-aggregated')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('$T_e$ [eV]')
# ax.set_ylim([1e-37,None])
ax.set_ylabel('Excitation radiation per ion [Wm$^3$]')
ax.legend()
ax.grid()
ax.set_title('Cooling curves: ' + el)

In [ ]:
# Plot radiative losses
PLT_nlj, PLT_nlj_eff = get_cooling_curves(r_nlj, el, kinetic=False)
Q_rad_nlj = 1e-6 * PLT_nlj_eff * ne * np.sum(r_nlj.impurities[el].dens_Max,1) * r_nlj.n_norm
PLT_nl, PLT_nl_eff = get_cooling_curves(r_nl, el, kinetic=False)
Q_rad_nl = 1e-6 * PLT_nl_eff * ne * np.sum(r_nl.impurities[el].dens_Max,1) * r_nl.n_norm
fig,ax = plt.subplots(1)
l, = ax.plot(r_nlj.xgrid * r_nlj.x_norm,Q_rad_nlj,color='black',label='j-resolved')
ax.plot(r_nl.xgrid * r_nlj.x_norm,Q_rad_nl,'--',color=l.get_color(), label='j-aggregated')
ax.set_xlabel('x [m]')
ax.set_ylabel('Excitation radiation [Wm$^{-3}$]')
ax.legend()
ax.grid()
ax.set_title('Total line radiation profiles: ' + el)
# ax.set_yscale('log')

In [5]:
from petsc4py import PETSc
import petsc4py

In [2]:
rhs = PETSc.Vec().createMPI(1000)
# rhs = PETSc.Vec().create(comm=None)

In [6]:
ksp = PETSc.KSP().create()

In [9]:
ksp.getTolerances()

(1e-05, 1e-50, 10000.0, 10000)

In [ ]:
rhs.assemblyBegin()
rhs.assemblyEnd()

In [ ]:
rhs.bindToCPU(0)

In [ ]:
help(PETSc.Mat().createAIJ)

In [ ]:
d1 = np.loadtxt('../gathered_dens_1_.txt')
d2 = np.loadtxt('../gathered_dens_2_.txt')
d4 = np.loadtxt('../gathered_dens_4_.txt')
d8 = np.loadtxt('../gathered_dens_8_.txt')

In [ ]:
fig,ax = plt.subplots(1)
x = 10
ax.plot(d1[x,:])
ax.plot(d2[x,:])
ax.plot(d4[x,:])
ax.plot(d8[x,:])
ax.set_yscale('log')